<a href="https://colab.research.google.com/github/RicardoMiles/Dissertation-template/blob/main/RandomSearch-parameter-202411221955.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/SummerProject/src')
!pwd

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/SummerProject/src


In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 定义包含CSV文件的文件夹路径
current_dir = os.getcwd()
root_dir = os.path.dirname(current_dir)
data_dir = os.path.join(root_dir, 'Data')
left_data = os.path.join(data_dir,'left')
right_data = os.path.join(data_dir,'right')

folder_path = left_data

# 获取left文件夹中所有CSV文件的列表
csv_files_left = [os.path.join(left_data, file) for file in os.listdir(left_data) if file.endswith('.csv')]
# 获取right文件夹中所有CSV文件的列表
csv_files_right = [os.path.join(right_data, file) for file in os.listdir(right_data) if file.endswith('.csv')]

print(f"Number of left files: {len(csv_files_left)}")
print(f"Number of right files: {len(csv_files_right)}")

Number of left files: 26
Number of right files: 26


In [6]:
from scipy import stats
# 定义特征提取函数
def extract_features(group):
    features = []
    # 遍历第三列到最后一列
    for col in group.columns[2:]:  # 从第三列开始提取特征
        data = group[col].values
        features.append(data.mean())  # 均值
        features.append(data.std())   # 标准差
        features.append(stats.skew(data))  # 偏度
        features.append(stats.kurtosis(data))  # 峰度
    return features

In [4]:
# 初始化存储所有特征和标签的列表
all_features = []
all_labels = []

# 初始化CSV文件计数器
csv_files_count = 0

# 处理left文件夹中的CSV文件
for file_name in csv_files_left:
    df = pd.read_csv(file_name)
    csv_files_count += 1  # 增加文件计数
    # 将数据按照epoch分组
    grouped = df.groupby(df.columns[1])  # 第二列是epoch分组的列
    for name, group in grouped:
        # 过滤掉time小于0的数据
        group = group[group[df.columns[0]] >= 0]  # 假设time列是第一列
        if len(group) == 81:  # 确保过滤后有81条数据
        # if len(group) == 101:
            # 提取特征
            features = group.iloc[:, 2:].values  # 第三列及以后是EEG通道信号
            all_features.append(features)
            # 获取标签
            all_labels.append('left')

# 处理right文件夹中的CSV文件
for file_name in csv_files_right:
    df_right = pd.read_csv(file_name)
    csv_files_count += 1  # 增加文件计数
    # 将数据按照epoch分组
    grouped_right = df_right.groupby(df_right.columns[1])  # 第二列是epoch分组的列
    for name, group in grouped_right:
        # 过滤掉time小于0的数据
        group = group[group[df_right.columns[0]] >= 0]  # 假设time列是第一列
        if len(group) == 81:  # 确保过滤后有81条数据
        # if len(group) == 101:
            # 提取特征
            features = group.iloc[:, 2:].values  # 第三列及以后是EEG通道信号
            all_features.append(features)
            # 获取标签
            all_labels.append('right')

# 将特征和标签转换为NumPy数组
features_array = np.array(all_features)
labels_array = np.array(all_labels)

# 打印处理的CSV文件数量
print(f"Total CSV files processed: {csv_files_count}")

Total CSV files processed: 52


In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

# 定义衡量分布相似性的函数
def evaluate_split_similarity(X_train, X_test):
    train_mean = X_train.mean(axis=0)
    test_mean = X_test.mean(axis=0)
    # 使用均方差衡量分布差异
    return np.mean((train_mean - test_mean) ** 2)

best_seed = None
min_difference = float('inf')

for seed in range(100):  # 遍历多个种子
    # 将 X 和 y 替换为 features_array 和 labels_array
    X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array, test_size=0.2, random_state=seed)
    diff = evaluate_split_similarity(X_train, X_test)
    if diff < min_difference:
        min_difference = diff
        best_seed = seed

print(f"Best seed: {best_seed}, Minimum difference: {min_difference}")


Best seed: 76, Minimum difference: 0.10239083541940588


In [5]:
print(f"Total samples: {features_array.shape[0]}")  # 样本数量
print(f"Timepoints per sample: {features_array.shape[1]}")  # 每个样本的时间点
print(f"Features per timepoint: {features_array.shape[2]}")  # 每个时间点的特征


Total samples: 7220
Timepoints per sample: 81
Features per timepoint: 101


In [6]:
from datetime import datetime
import tensorflow as tf

np.random.seed(76)
tf.random.set_seed(76)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array, test_size=0.2, random_state=42)

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

Current processing time: 2024-11-22 17:47:54


In [7]:
print(f"Initial X_train shape: {X_train.shape}")
print(f"Initial X_test shape: {X_test.shape}")

Initial X_train shape: (5776, 81, 101)
Initial X_test shape: (1444, 81, 101)


In [8]:
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 初始化标签编码器
label_encoder = LabelEncoder()

# 将训练集和测试集的标签转换为数值
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# 确保转换后的标签是正确的数据类型
y_train_encoded = y_train_encoded.astype('int32')
y_test_encoded = y_test_encoded.astype('int32')

In [12]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, InputLayer
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
!pip install keras-tuner
import keras_tuner as kt
from datetime import datetime

# 假设 X_train, X_test, y_train, y_test 已经定义并准备好了

def build_model(hp):
    model = Sequential()
    model.add(InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])))

    # LSTM layers
    for i in range(hp.Int('lstm_layers', 1, 3)):
        model.add(LSTM(units=hp.Int('units_' + str(i), min_value=10, max_value=100, step=10),
                       return_sequences=i < hp.Int('lstm_layers', 1, 3) - 1))
        model.add(Dropout(hp.Float('dropout_' + str(i), min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))  # 二分类问题

    # Compile model
    model.compile(optimizer=Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=100,  # 你可以根据需要调整试验次数
    executions_per_trial=1,
    directory='my_dir',
    project_name='intro_to_kt'
)

tuner.search(X_train, y_train_encoded, epochs=35, batch_size=32, validation_data=(X_test, y_test_encoded))

# 获取最佳模型
best_model = tuner.get_best_models(num_models=1)[0]

# 评估模型
loss, accuracy = best_model.evaluate(X_test, y_test_encoded, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

Trial 100 Complete [00h 01m 31s]
val_accuracy: 0.5491690039634705

Best val_accuracy So Far: 0.5706371068954468
Total elapsed time: 01h 52m 45s


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Test Loss: 0.6870168447494507
Test Accuracy: 0.5706371068954468
Current processing time: 2024-11-22 19:43:59


In [9]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, InputLayer
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

# 假设 X_train, X_test, y_train, y_test 已经定义并准备好了
# 构建LSTM模型
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.4))  # 添加Dropout层
model.add(LSTM(units=30))
model.add(Dropout(0.4))  # 添加Dropout层
model.add(Dense(1, activation='sigmoid'))  # 二分类问题

# 编译模型
model.compile(Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# 早停法
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# 训练模型
model.fit(X_train, y_train_encoded, epochs=35, batch_size=32, validation_data=(X_test, y_test_encoded))

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.4955 - loss: 0.7119 - val_accuracy: 0.5014 - val_loss: 0.6976
Epoch 2/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5053 - loss: 0.7061 - val_accuracy: 0.5145 - val_loss: 0.6951
Epoch 3/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5293 - loss: 0.6951 - val_accuracy: 0.5145 - val_loss: 0.6941
Epoch 4/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5272 - loss: 0.6922 - val_accuracy: 0.5173 - val_loss: 0.6938
Epoch 5/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5365 - loss: 0.6899 - val_accuracy: 0.5132 - val_loss: 0.6934
Epoch 6/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5373 - loss: 0.6912 - val_accuracy: 0.5145 - val_loss: 0.6927
Epoch 7/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5296 - loss: 0.6910 - val_accuracy: 0.5215 - val_loss: 0.6926
Epoch 8/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5463 - loss: 0.6864 - val_accur

In [13]:
best_hps = tuner.get_best_hyperparameters()[0]
for key, value in best_hps.values.items():
    print(f"{key}: {value}")

lstm_layers: 2
units_0: 30
dropout_0: 0.5
learning_rate: 0.005283071526371937
units_1: 80
dropout_1: 0.1
units_2: 10
dropout_2: 0.2


In [14]:
# 预测测试集
predictions = model.predict(X_test)

# 将概率转换为类别
predicted_classes = (predictions > 0.5).astype("int32")

# 打印预测结果
print(predicted_classes)

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
[[1]
 [0]
 [0]
 ...
 [1]
 [1]
 [0]]
Current processing time: 2024-11-22 19:55:03


In [15]:
# 计算精确度、召回率和F1分数
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

precision = precision_score(y_test_encoded, predicted_classes)
recall = recall_score(y_test_encoded, predicted_classes)
f1 = f1_score(y_test_encoded, predicted_classes)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# 计算混淆矩阵
cm = confusion_matrix(y_test_encoded, predicted_classes)
print("Confusion Matrix:")
print(cm)

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

Precision: 0.48625180897250364
Recall: 0.4827586206896552
F1 Score: 0.4844989185291997
Confusion Matrix:
[[393 355]
 [360 336]]
Current processing time: 2024-11-22 19:55:05
